# Day 19

In [1]:
import numpy as np
import cvxpy as cp
import re

ModuleNotFoundError: No module named 'scipy.sparse.linalg.eigen.arpack'; 'scipy.sparse.linalg.eigen' is not a package

## Part 1

In [44]:
def optimize_blueprint(costs, time):
    ore_ore, clay_ore, obsidian_ore, obsidian_clay, geode_ore, geode_obsidian = costs
    cost_matrix = np.array([
        [ore_ore, clay_ore, obsidian_ore, geode_ore],
        [0, 0, obsidian_clay, 0],
        [0, 0, 0, geode_obsidian],
        [0, 0, 0, 0]
    ])

    # resource_variables[i] = how many of each resource available after minute i. resource_variables[i-1] - (costs * production_variables[i]) + robot_variables[i]
    # robot_variables[i] = how many robots are available on minute i to work. essentially robot_variables[i-1] + production_variables[i-1]
    # production_variables[i] = how many of each robot to build. can only build one per minute, restricted to using resource_variables[i-1].
    resource_variables = []
    robot_variables = []
    production_variables = []
    constraints = []
    for i in range(time + 1):
        resources = cp.Variable(4)
        robots = cp.Variable(4)
        production_decisions = cp.Variable(4, boolean=True)

        if i == 0:
            constraints.append(resources == 0)
            constraints.append(robots[0] == 1)
            constraints.append(robots[1:] == 0)
            constraints.append(production_decisions == 0)
        else:
            prev_resources = resource_variables[-1]
            constraints.append(robots == robot_variables[-1] + production_variables[-1])
            
            constraints.append(cp.sum(production_decisions) <= 1)

            constraints.append(cost_matrix @ production_decisions <= prev_resources)
            constraints.append(resources == (prev_resources + robots - (cost_matrix @ production_decisions)))

        robot_variables.append(robots)
        resource_variables.append(resources)
        production_variables.append(production_decisions)

    objective = cp.Maximize(resource_variables[-1][3])
    problem = cp.Problem(objective, constraints)
    problem.solve(solver='ECOS_BB')
    return problem.value

In [45]:
def parse_line(line:str)->list[int]:
    blueprint_index = int(line.split('.')[0].split(':')[0].split('Blueprint ')[1])
    ore_ore = int(line.split('.')[0].split(':')[1].split(' ore')[1].split('costs ')[1])
    clay_ore = int(line.split('.')[1].split(' ore')[0].split('costs ')[1])
    obsidian_ore = int(line.split('.')[2].split('costs ')[1].split(' ore and ')[0])
    obsidian_clay = int(line.split('.')[2].split('costs ')[1].split(' ore and ')[1].split(' clay')[0])
    geode_ore = int(line.split('.')[3].split('costs ')[1].split(' ore and ')[0])
    geode_obsidian = int(line.split('.')[3].split('costs ')[1].split(' ore and ')[1].split(' obsidian')[0])
    return [blueprint_index, ore_ore, clay_ore, obsidian_ore, obsidian_clay, geode_ore, geode_obsidian]

In [46]:
with open('test_day19.txt') as input_text:
    total_quality = 0
    for line in input_text:
        if line != '\n':
            vals = parse_line(line)
            blueprint_index, costs = numbers[0], numbers[1:]
            print(optimize_blueprint(costs,time=24))
            #total_quality += bp_id * optimize_blueprint(costs, time=24)
            break
    print(total_quality)

11.00000000015008
0


/Users/daniel.marthaler/.pyenv/versions/3.9.10/envs/cmdstanpy/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [37]:
ore_ore, clay_ore, obsidian_ore, obsidian_clay, geode_ore, geode_obsidian = all_costs[0][1:]
cost_matrix = np.array([
    [ore_ore, clay_ore, obsidian_ore, geode_ore],
    [0, 0, obsidian_clay, 0],
    [0, 0, 0, geode_obsidian],
    [0, 0, 0, 0]
])

In [38]:
cost_matrix

array([[ 4,  2,  3,  2],
       [ 0,  0, 14,  0],
       [ 0,  0,  0,  7],
       [ 0,  0,  0,  0]])

## Part 2

In [ ]:
import numpy as np
import cvxpy as cp
import re



with open('input') as f:
    blueprints = f.readlines()
    total_quality = 0
    for blueprint in blueprints:
        numbers = list(map(int, re.findall('\d+', blueprint)))
        bp_id, costs = numbers[0], numbers[1:]
        total_quality += bp_id * optimize_blueprint(costs, time=24)
    print(total_quality)

with open('input') as f:
    blueprints = f.readlines()
    total_quality = 1
    for blueprint in blueprints[:3]:
        numbers = list(map(int, re.findall('\d+', blueprint)))
        bp_id, costs = numbers[0], numbers[1:]
        total_quality *= optimize_blueprint(costs, time=32)
    print(total_quality)